<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Находим-F1-на-тесте" data-toc-modified-id="Находим-F1-на-тесте-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Находим F1 на тесте</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

`Описание проекта:` в этом проекте мы созадим модель, которая будеть проверять текстовые комментарии на цензурность и токсичность, классифицируя тот или иной комментарий соответствующим образом.

## Подготовка

Импортируем библиотеки согласно PEP-8

In [17]:
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [11]:
import numpy as np
import pandas as pd

import catboost as cb
from catboost import CatBoostClassifier
from itertools import product
import lightgbm as lgb
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
#from pymystem3 import Mystem
from tqdm import notebook
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...


True

Загружаем данные

In [3]:
try: 
    df = pd.read_csv('toxic_comments.csv', index_col=[0])
except: 
    df = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0])

Создаём переменную для вывода информации о данных

In [4]:
def show_i(data):

    '''Выводим на экран следущую информацию:

    - размер таблицы
    - основную информацию
    - первые 5 строк таблицы
    - суммарная статистика по числовым параметрам
    '''

    display(data.shape)
    display(data.info())
    display(data.head())
    display(data.describe())

In [5]:
show_i(df)

(159292, 2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


None

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Ищем пропуски

In [6]:
# Выводим таблицу с пропусками в процентах

pd.DataFrame(round(df.isna().mean()*100)).style.background_gradient('coolwarm')

,0
text,0.000000
toxic,0.000000


`Вывод:` в датасете содержится 159292 строк и два признака: текст и целевой признак - токсичность текста.

пропусков нет.

Готовим данные для обучения

In [7]:
# делим на обучающую и тестовые выборки
train, test = train_test_split(df.dropna(), test_size=0.1)

print(train.shape)
print(test.shape)

(143362, 2)
(15930, 2)


Создаём два корпуса текстов: состоящие из обучающей и тестовой выборки

In [9]:
# Создаём два корпуса текстов: состоящие из обучающей и тестовой выборки

corpus = train['text'].values.astype(str)
corpus_test = test['text'].values.astype(str)

In [19]:
#  NLTK 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Инициализация  WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# применяем лемматизацию
def lemmatize_with_pos(text):
    pos_mapping = {
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV,
        'J': wordnet.ADJ
    }
    # токенизация текста
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    
    # лемматизация с POS
    lemmatized_words = []
    for word, pos in pos_tags:
        wordnet_pos = pos_mapping.get(pos[0].upper(), wordnet.NOUN)
        lemmatized_word = lemmatizer.lemmatize(word, pos=wordnet_pos)
        lemmatized_words.append(lemmatized_word)
    
    # объединение лемматизированных слов в строку
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

# применяем функцию
df['text'] = df['text'].apply(lemmatize_with_pos)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Лемматизируем обучающий и тестовы корпуса

In [60]:
# Лемматизируем корпуса с применением функции

corpus = lemmatize(corpus[0])
corpus_test = lemmatize(corpus_test[0])

Скачиваем и сохраняем стоп-слова на английском

In [61]:
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение

Приступаем к обучению модели

In [63]:
train.head()

,text,toxic
113737,"""\n\n ZOG Mediation \n\n(By the way, you seem ...",0
122320,"It's a full featured program, so I doubt that ...",0
146539,I was afraid that might come up when I saw you...,0
15899,Off-the-cuff remarks very briefly: this articl...,0
58839,Thanks for the bold edits! With my limited gr...,0


In [64]:

corpus = train['text'].tolist()
corpus_test = test['text'].tolist()

# Создаём объект TfidfVectorizer с английскими стоп-словами
tf_idf = TfidfVectorizer(stop_words=stopwords)

# Трансформируем обучающую выборку в матрицу
features_train = tf_idf.fit_transform(corpus)
target_train = train['toxic']

# Трансформируем тестовую выборку в матрицу
features_test = tf_idf.transform(corpus_test)
target_test = test['toxic']


Обучаем модель Логистической Регрессии

In [66]:
%%time
# Определяем диапазон гиперпараметров для Grid Search
param_grid = {
    
    'C': [1.0, 10.0],  # Параметры регуляризации
    'penalty': ['l1', 'l2'],  # Тип регуляризации (L1 или L2)
    
}

# Создаём модель логистической регрессии
logreg = LogisticRegression(random_state=12345, class_weight='balanced')

# инициализация Grid Search с кроссвалидацией и перебором гиперпараметров
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='f1')

# обучаем модель на обучающей выборке
grid_search.fit(features_train, target_train)

print('Best parameters:', grid_search.best_params_)
print('F1 score:', grid_search.best_score_)

# Получаем лучшие параметры для логистической регрессии
best_params_logreg = grid_search.best_params_

c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

Best parameters: {'C': 10.0, 'penalty': 'l2'}
F1 score: 0.7672128548653958
CPU times: total: 2min 35s
Wall time: 1min 24s


c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


`Вывод:` по итогу проверки на тренировочных данных на модели Логистической Регрессии, были выявлены самые лучшие гипрепараметры:

- 'C': 10.0
- 'penalty': 'l2'

F1 score: 0.7672128548653958

Random Forest

In [67]:
%%time
param_grid = {
    'n_estimators': np.arange(20),
    'min_samples_split': np.arange(2, 10, 2),
    'min_samples_leaf': np.arange(1, 10, 1),
    'max_features': ['sqrt','log2']
}

# создаём и сохраняем в переменную модель RandomForestClassifier
estimator = RandomForestClassifier(random_state=12345)

# инициализация RandomizedSearchCV с кроссвалидацией и перебором гиперпараметров
random_search = RandomizedSearchCV(estimator, param_grid, scoring='f1', n_iter=10, cv=5, random_state=12345)
random_search.fit(features_train, target_train)

# находим и сохраняем предсказания илучшей модели
predictions = random_search.best_estimator_.predict(features_test)

# выводим лучшие параметры и лучшую F1 метрику
print("Best estimator:", random_search.best_estimator_)
print("F1:", random_search.best_score_)

Best estimator: RandomForestClassifier(min_samples_leaf=2, min_samples_split=6, n_estimators=9,
                       random_state=12345)
F1: 0.6401141566901527
CPU times: total: 39min 49s
Wall time: 41min 1s


`Вывод:` по итогу проверки на тренировочных данных на модели Random Forest, были выявлены самые лучшие гипрепараметры:

- min_samples_leaf=2
- min_samples_split=6
- n_estimators=9
- random_state=12345

F1: 0.6401141566901527

LightGBM

In [68]:
%%time
# задаём параметры
params = {
    'objective': ['regression'],
    'metric': ['f1'],
    'boosting_type': ['gbdt'],
    'learning_rate': [0.05],
    'feature_fraction': [0.9],
    'reg_alpha': [0.1, 0.5],
    }

# сохраняем модель в переменную
model = lgb.LGBMClassifier()

# производим перебор параметров и обучение модели
grid_search = GridSearchCV(model, params, cv=5, scoring='f1')
grid_search.fit(features_train, target_train)

# выводим лучшие параметры и метрику RMSE
print('Best parameters:', grid_search.best_params_)
print('F1 score:', grid_search.best_score_)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGB

`Вывод:` по итогу проверки на тренировочных данных на модели LightGBM, были выявлены самые лучшие гипрепараметры:

- 'boosting_type': 'gbdt', 
- 'feature_fraction': 0.9, 
- 'learning_rate': 0.05, 
- 'metric': 'f1', 
- 'objective': 'regression', 
- 'reg_alpha': 0.1

F1 score: 0.7097324902763346

`Общий вывод:` самой лучшей моделью с наилучшей метрикой F1 является модель логистической регрессии с гиперпараметрами:


- 'C': 10.0
- 'penalty': 'l2'

Применим её для обучения и проверки на тестовой выборке

#### Находим F1 на тесте

In [69]:
# Создаём модель логистической регрессии с лучшими параметрами
logreg_best = LogisticRegression(random_state=12345, class_weight='balanced', **best_params_logreg)

# обучаем модель с лучшими параметрами на обучающей выборке
logreg_best.fit(features_train, target_train)

# делаем предсказания
test_predictions = logreg_best.predict(features_test)

# вычисляем F1-метрику на тестовой выборке
test_f1_score = f1_score(target_test, test_predictions)
print('F1 score on test data:', test_f1_score)

F1 score on test data: 0.7670842824601367


c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


`Вывод:` на тесте наша лучшая модель логистической регресии показала приемлимый результат F1 score on test data: 0.7670842824601367 что выше порогового значения 0.75.

## Выводы

Целью данного проекта было создане модели которая сможет классифицировать текстовые комментарии и классифицировать их на токсичность.

Для машинной обработки, тексты были преобразованы в вектора. Для этого были сосзданы корпуса слов и стоп-слова.

Для обучения были использованы следующие модели:

- Логистическая регрессия
- Случайный лес
- LightGBM

По результатам тестирования, лучшей моделью оказалась - Логистическая регрессия со след. параметрами:

- 'C': 10.0
- 'penalty': 'l2'

Результат метрики F1 на тестовой выборке показал 0.7670842824601367 что выше порогового значения 0.75.

